In [1]:
import json

SPLIT_DATASET = 1

In [2]:
with open ('./frames.json', "r") as f:
    data = json.loads(f.read())

In [3]:
def get_utterances():

    keys = ['count_dst_city', 'end_date', 'n_children',
            'dst_city', 'breakfast', 'seat', 'or_city',
            'name', 'price', 'beach', 'n_adults', 'min_duration',
            'category', 'university', 'budget', 'intent',
            'str_date', 'max_duration', 'park', 'flex']

    interest_keys = ['end_date',
                     'dst_city', 
                     'or_city',
                     'budget',
                     'str_date']
    
    clean_keys = {'end_date': 'end_date',
                  'dst_city': 'to', 
                  'or_city': 'from',
                  'budget': 'budget',
                  'str_date': 'start_date'}

    uts = []
    
    for text_id in range(len(data)):
        text = data[text_id]['turns'][0]['text']
        if len(text.split())>2:
            tmp_ut = {"text": text,
                      "intent": "Find trip",
                      "entities": []}
            for val_key_list_dict in data[text_id]['turns'][0]['labels']['acts_without_refs']:
                val_key_list = val_key_list_dict['args']
                for val_key in val_key_list:
                    if val_key['key'] in interest_keys and val_key['val'] is not None:
                        tmp_index = text.find(val_key['val'])
                        if tmp_index != -1:
                            if val_key['key'] == 'end_date' or val_key['key'] == 'str_date':
                                tmp_entity = {"entity": "datetimeV2",
                                              "role": clean_keys[val_key['key']]}
                            else:
                                tmp_entity = {"entity": clean_keys[val_key['key']]}
                            tmp_entity["startPos"] = tmp_index
                            tmp_entity["endPos"] = tmp_index + len(val_key['val']) - 1
                            tmp_entity["children"] = []
                            tmp_ut["entities"].append(tmp_entity)
            if len(tmp_ut["entities"]):
                uts.append(tmp_ut)
                          
    return uts

In [4]:
with open("test_batch.json", "w") as f:
        json.dump(get_utterances()[:100], f, indent=2)

In [5]:
param_dict_1 = {
  "luis_schema_version": "7.0.0",
  "intents": [
    {
      "name": "Find trip",
      "features": []
    }
  ],
  "entities": [
    {
      "name": "budget",
      "children": [],
      "roles": [],
      "features": []
    },
    {
      "name": "from",
      "children": [],
      "roles": [],
      "features": []
    },
    {
      "name": "to",
      "children": [],
      "roles": [],
      "features": []
    }
  ],
  "hierarchicals": [],
  "composites": [],
  "closedLists": [],
  "prebuiltEntities": [
    {
      "name": "datetimeV2",
      "roles": [
        "end_date",
        "start_date"
      ]
    }
  ]}
  

param_dict_2 = {"versionId": "1.0",
  "name": "CustomTripFinding",
  "desc": "A LUIS model to extract cities of origin and destination, dates and budget.",
  "culture": "en-us",
  "tokenizerVersion": "1.0.0",
  "patternAnyEntities": [],
  "regex_entities": [],
  "phraselists": [],                
  "regex_features": [],
  "patterns": [],
  "settings": []}


def make_json(prms_dct_1, prms_dct_2):
    if SPLIT_DATASET:
        training_data = {"utterances": get_utterances()[100:]}
    else:
        training_data = {"utterances": get_utterances()}
    output = {**prms_dct_1, **training_data, **prms_dct_2}
    with open("CustomTripFinding.json", "w") as f:
        json.dump(output, f, indent=2)


In [6]:
make_json(param_dict_1, param_dict_2)